## Potential Pipeline Process

Process that:

1. fetches building footprints from OSM (OSMnx)

2. reads your 15-minute demand tables from Supabase and computes annual energy need

3. downloads GeoTIFFs from Supabase to compute a simple shade index per building

4. estimates roof orientation, tilt, area, and per-building annual solar energy potential

5. computes a composite solar_suitability score for each building

6. selects a set of buildings (residential vs commercial) to meet the user’s chosen percentage of city power from solar while trying to respect the chosen commercial/building mix

7. visualizes everything on a PyDeck map (colored polygons by solar_score and highlighted chosen buildings)

8. writes results back to Supabase as GeoJSON in a building_suitability table

Modularized so we can swap in better irradiance, panel efficiency, or LIDAR later.

In [ ]:
# Imports
#pip install streamlit supabase osmnx geopandas rasterio shapely pyproj pydeck
# all possibles so far, may need to add more as necessary
import streamlit as st
from supabase import create_client, Client
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import tempfile
import rasterio
from rasterio.mask import mask
import osmnx as ox
from shapely.geometry import Polygon, mapping
import pydeck as pdk
import json
from datetime import datetime, timedelta

In [ ]:
st.set_page_config(layout="wide", page_title="Solar Suitability Planner")

# This is how I set up the secrets on my machine, not the same as Postgres!
url = st.secrets["SUPABASE_URL"]
key = st.secrets["SUPABASE_KEY"]
supabase: Client = create_client(url, key)

# Default average daily insolation in kWh/m2/day (tweakable)
DEFAULT_INSOLATION = {
    "Ann Arbor": 4.0,   # ~kWh/m2/day 
    "Tucson": 6.0,
}
PANEL_EFFICIENCY = 0.18   # guessing, can replace from PV avgs
PERFORMANCE_RATIO = 0.75  # guessing, can replace from PV avgs

In [ ]:
# WORKING
def load_demand_table(city: str) -> pd.DataFrame:
    """
    Load city demand table from Supabase for selected city
    Expected numeric column: MW and datetime column named 'date_time'.
    """
    table_name = "Ann_Arbor_demand" if city == "Ann Arbor" else "TEPC_demand"
    res = supabase.table(table_name).select("*").execute()
    data = res.data
    df = pd.DataFrame(data)
    return df

In [ ]:
# WORKING
# Takes a while to run: 3m 13s for Tucson!
def fetch_buildings_osm(place_name: str) -> tuple:
    """
    Use OSMnx to fetch building footprints for the given place name.
    Returns GeoDataFrame with area_m2 computed per building and total area.
    """
    tags = {"building": True}
    gdf = ox.features.features_from_place(place_name, tags=tags)
    
    # Filter to only Polygons and MultiPolygons
    gdf = gdf[gdf.geometry.type.isin(['Polygon', 'MultiPolygon'])]
    
    # Residential/Commercial Classification
    gdf["is_commercial"] = gdf.apply(
        lambda row: (
            (pd.notna(row.get("building")) and str(row.get("building")).lower() in
             ["commercial", "retail", "industrial", "office", "warehouse"])
        ), axis=1
    )

    # Calculate the area (meters)
    gdf_proj = ox.projection.project_gdf(gdf)
    gdf['area_m2'] = gdf_proj.area

    # Total area (sq m) covered by building footprints - this will go to City Specs
    total_area = gdf['area_m2'].sum()

    return gdf, print(total_area)   # remove print() for production

In [ ]:
# For using TIFs to obtain shading, tilt, etc
# Need to set up paths correctly and make sure all images in buckets

def download_geotiff_from_supabase(bucket: str, path: str) -> rasterio.io.DatasetReader:
    """
    Download a file from Supabase storage bucket to a temp file and open with rasterio.
    """

In [ ]:
def shade_from_geotiff(raster: rasterio.io.DatasetReader, polygon: Polygon):
    """
    Compute a simple shade score (0 bright / 1 shaded) by masking the raster to the polygon and computing mean brightness.
    Assumes first band is usable (grayscale or visible).
    """

In [ ]:
# WORKING
def compute_city_annual_kwh(df: pd.DataFrame):
    """
    Convert the 15-minute MW measurements into annual kWh.
    Each MW measurement over a 15-min interval contributes: MW * 0.25 hours = MWh.
    Sum of (MW) * 0.25 gives MWh per dataset period; multiply by 1000 to kWh.
    """
    # For each year: sum(MW) * 0.25 hours = MWh -> convert to kWh
    summary = (
        df.groupby('Year')['MW']
        .sum()
        .reset_index(name='total_MW_sum')
    )
    summary['annual_kWh'] = summary['total_MW_sum'] * 0.25 * 1000

    summary = summary.sort_values("Year").reset_index(drop=True)
    summary["growth_rate"] = summary["annual_kWh"].pct_change()

    last_year = summary.iloc[-1]["Year"]
    last_demand = summary.iloc[-1]["annual_kWh"]
    last_growth = summary.iloc[-1]["growth_rate"]

    predicted = last_demand * (1 + last_growth)

    return print(predicted)   # remove print for production

In [ ]:
# May be able to calculate using saved Azimuth values and MATH
# Don't need to discard any, we can set a limit for unacceptable in the 
# select_buildings_to_meet_target()

def orientation_match_score(roof_angle_deg: float, ideal_sun_azimuth_deg: float):
    """
    Compute orientation score for optimal cardinal directions in degrees
    """
    diff = abs(roof_angle_deg - ideal_sun_azimuth_deg) % 360
    if diff > 180:
        diff = 360 - diff
    # map 0->1, 180->0
    return 1.0 - (diff / 180.0)

In [ ]:
# WORKING
def polygon_orientation(polygon: Polygon):
    """
    Estimate roof orientation (degrees clockwise from North) by minimum_rotated_rectangle method.
    Returns degrees 0-360  
    We'll return as degrees clockwise from North:
    convert from arctan2(dx, dy) with adjustments.
    """
    if polygon.is_empty:
        return np.nan
    mrr = polygon.minimum_rotated_rectangle
    coords = list(mrr.exterior.coords)
    max_len = 0
    best_angle = None
    # iterate edges
    for i in range(len(coords)-1):
        x1,y1 = coords[i]
        x2,y2 = coords[i+1]
        dx = x2 - x1
        dy = y2 - y1
        edge_len = np.hypot(dx, dy)
        if edge_len > max_len:
            max_len = edge_len
            best_angle = np.degrees(np.arctan2(dy, dx))
    if best_angle is None:
        return np.nan
    # best_angle is angle from +x axis (East) CCW. Convert to degrees clockwise from North:
    # angle_from_north_clockwise = (90 - best_angle) mod 360
    angle = (90 - best_angle) % 360
    return angle

In [ ]:
# WORKING
# OG gdf from osmnx has 350+ columns; possibly change building acquisition 
# to limit to only necessary, might improve speed as well. 
# Can update with .txt containing column list with ideal columns marked
def estimate_tilt_from_height_and_span(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Estimate roof tilt in degrees using any available height / building:levels

    - If 'height' available (meters), use it
    - Else if 'building:levels' available, multiply by 3.2
    - Else use small heights with default tilt for commercial vs residential
    """
    # handle numeric heights
    if "height" in gdf.columns:
        heights = pd.to_numeric(gdf["height"], errors="coerce")
    elif "building:levels" in gdf.columns:
        heights = pd.to_numeric(gdf["building:levels"], errors="coerce") * 3.2
    else:
        heights = pd.Series(np.nan, index=gdf.index)

    # Effective roof span (approx): use sqrt(area) or MRR longest edge-derived width
    def approx_width(geom):
        try:
            mrr = geom.minimum_rotated_rectangle
            # approximate width by dividing perimeter by 4 (rectangle assumption)
            perim = mrr.length
            width = perim / 4.0
            return max(width, np.sqrt(geom.area))
        except Exception:
            return np.sqrt(geom.area)

    gdf["width_m"] = gdf.geometry.to_crs(epsg=3857).apply(lambda geom: approx_width(geom))
    heights_filled = heights.fillna(5.0)
    # tilt = arctan(height / (width/2)) [approx roof pitch]
    gdf["tilt_deg"] = np.degrees(np.arctan2(heights_filled, gdf["width_m"]/2.0))
    # clamp sensible range
    gdf["tilt_deg"] = gdf["tilt_deg"].clip(3, 45)
    return gdf

In [ ]:
# WORKING
# shade, orientation and tilt (not mathmatical) need to come from rasters and GeoTIFFs
def compute_suitability_scores(gdf: gpd.GeoDataFrame, irradiance_factor: float, ideal_sun_azimuth: float):
    """
    Calculate final solar_score from area, and if possible: orientation, tilt, shade, and irradiance.
    Compute intermediate scores in [0,1] and combine with weights.
    """
# orientation
    gdf["orientation_deg"] = gdf.geometry.apply(lambda geom: polygon_orientation(geom))
    gdf["orientation_score"] = gdf["orientation_deg"].apply(lambda a: orientation_match_score(a, ideal_sun_azimuth))
    # tilt: ideal ~25 deg (residential) -> score drops as further away
    gdf["tilt_score"] = 1.0 - (np.abs(gdf["tilt_deg"] - 25.0) / 40.0)
    gdf["tilt_score"] = gdf["tilt_score"].clip(0,1)
    # irradiance_factor applied uniformly (0..1)
    irr = float(np.clip(irradiance_factor, 0.0, 1.0))
    # combine: weights can be tuned
    gdf["solar_score"] = (
        0.35 * gdf["orientation_score"] +
        0.25 * gdf["tilt_score"] +
        0.15 * irr
    )
    gdf["solar_score"] = gdf["solar_score"].clip(0,1)
    return gdf

In [ ]:
# WORKING - BUT!
# Results still need tweaking
# How many panels per usable area?
# Num hours of daylight - plug in sunrise/sunset
# Can't add snow factor, no way to tie in without lat/long
# Toronto: yield = 16,417 / 14 = 1,172 kWh/kW/year
# will need azimuth for more accuracy
# use estimated yield per location to identify overall loss factor (since we can't add all the realistic ones)
#
# Ex from SolarTO: 
# System size:14 kw, 
# Annual electricity generation:16,417 kwh, 
# Roof size suitable for solar: 3,230.88 m2 (10,600 sq ft)
#
# This says 
# Annual elctricity is: 84,8604 kwh
# Roof size: 2,870.3 m2 roof
def estimate_building_annual_potential_kwh(gdf: gpd.GeoDataFrame, insolation_kwh_m2_day: float,
                                           panel_efficiency=PANEL_EFFICIENCY, perf_ratio=PERFORMANCE_RATIO):
    """
    Estimate the annual kWh each building can produce.
    """
   # Set limit on useable area
    COMMERCIAL_FRACTION = 0.50
    RESIDENTIAL_FRACTION = 0.25 

    # Compute usable area
    gdf["usable_fraction"] = gdf["is_commercial"].apply(
        lambda x: COMMERCIAL_FRACTION if x else RESIDENTIAL_FRACTION
    )
    gdf["usable_area_m2"] = gdf["area_m2"] * gdf["usable_fraction"]

    # Energy factor based on insolation + efficiency
    daily = float(insolation_kwh_m2_day)
    factor = daily * 365.0 * panel_efficiency * perf_ratio
    gdf["annual_potential_kwh"] = gdf["usable_area_m2"] * factor

    return gdf

In [ ]:
def select_buildings_to_meet_target(gdf: gpd.GeoDataFrame, required_kwh: float, commercial_pct: float):
    """
    Greedy selection that alternates choosing commercial/residential in a ratio
    that tries to match the requested commercial_pct while picking highest solar_score first.
    Returns GeoDataFrame of selected buildings and remaining totals.
    """

In [ ]:
# Results summary

In [ ]:
# Save geometry results to Supabase

Notes:

-might use LiDAR (DEM) for true solar path shading

-currently using default irradiance values (Ann Arbor ~4, Tucson ~6 kWh/m²/day), but maybe can be replaced with city-specific measured irradiance (NREL) for better accuracy

-add an endpoint to export selected parcels as GeoJSON/CSV

-store results back in Supabase (then the app only reads results)